In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [3]:
db = "DB_CS250_Snapshot"
users = pd.read_json(f"{db}/users/documents.jsonl", lines=True)
logs = pd.read_json(f"{db}/logs/documents.jsonl", lines=True)
exercises = pd.read_json(f"{db}/exercises/documents.jsonl", lines=True)
weeks = pd.read_json(f"{db}/weeks/documents.jsonl", lines=True)
weeks=weeks[weeks["name"]!="Playground for the first three Quizzes"]


In [4]:

logs["exerciseName"] = logs["exerciseId"].apply(lambda x: exercises[exercises["_id"]==x]["name"].values[0])
logs["controlGroup"] = logs["exerciseId"].apply(lambda x: exercises[exercises["_id"]==x]["controlGroup"].values[0])
logs["userGroup"] = logs["userId"].apply(lambda x: users[users["_id"]==x]["group"].values[0])
logs["_creationTime"] = pd.to_datetime(logs["_creationTime"], unit="ms")
#exercise week
exo_week = exercises[["_id","weekId"]].merge(weeks[["_id","name"]].rename(columns={"_id":"weekId"}),on="weekId")

logs["week"] = logs["exerciseId"].apply(lambda x: next(iter(exo_week[exo_week["_id"]==x]["name"].values), "invalid"))


In [5]:
logs["week"].value_counts()

week
Quiz #5     15970
Quiz #6     15631
Quiz #4     14521
Quiz #8     14462
Quiz #3     14413
Quiz #1     13923
Quiz #2     13130
Quiz #11    12850
Quiz #7     12315
Quiz #10    12017
Quiz #9     11103
invalid      3634
Testing        33
Name: count, dtype: int64

In [6]:
valid_weeks = [f'Quiz #{i}' for i in range(1,12)]
logs = logs[logs["week"].isin(valid_weeks)]
print(logs["week"].value_counts())

week
Quiz #5     15970
Quiz #6     15631
Quiz #4     14521
Quiz #8     14462
Quiz #3     14413
Quiz #1     13923
Quiz #2     13130
Quiz #11    12850
Quiz #7     12315
Quiz #10    12017
Quiz #9     11103
Name: count, dtype: int64


In [7]:
quiz_started = logs[logs['type'] == "quizStarted"].drop_duplicates(subset=["userId", "exerciseName"],keep='last')
correct_submissions = logs[(logs['type'] == "quizSubmission") & 
                        (logs['details'].apply(lambda x: isinstance(x, dict) and x.get('correctness') == 1))]

logs_grouped_quiz = pd.concat([quiz_started, correct_submissions])
logs_grouped_quiz["timeSpent_quiz"] = logs_grouped_quiz.groupby(["userId", "exerciseName"])[["_creationTime"]].diff()


logs_grouped_quiz["timeSpent_quiz"] = logs_grouped_quiz["timeSpent_quiz"].apply(lambda x: x.total_seconds()/60)
logs_grouped_quiz = logs_grouped_quiz[["userId","exerciseId", "timeSpent_quiz"]]


In [8]:
logs_ts = logs.merge(logs_grouped_quiz.dropna(subset='timeSpent_quiz'), how='left',on=['userId','exerciseId'])
logs_ts.head()

,_creationTime,_id,attemptId,exerciseId,type,userId,variant,details,systemMessageId,userMessageId,exerciseName,controlGroup,userGroup,week,timeSpent_quiz
0,2024-05-11 15:56:52.688503174,k570005nhp144860betwtwy7v56rx64m,j571kc5p8ah7yvj1xhng7nzpgs6rxy2m,k179wpzex0za69rx2wyv02e9hd6rj4fb,attemptStarted,js71tk2nax2wvs97m3vgc5erp16mds09,reading,NaN,NaN,NaN,Kruskal's algorithm,A,A,Quiz #9,1.769433
1,2024-04-16 08:00:01.116921631,k57000a4rv5wgy13d63pqyt51s6qb2vv,j57adjsev6qjazakv9p4ad7qwh6qany3,k1710bmgh2mwgaqv7sjkmmfh1x6pveqf,quizSubmission,js7ese808vctnr15sy1te2pvx96mndmx,explain,"{'answers': [2.0, 0.0, 2.0], 'correctness': 1....",NaN,NaN,Optimal Binary Search Trees,A,B,Quiz #5,18.036219
2,2024-05-20 16:28:14.092592285,k57000d308fpr45yqrprf6wrs56see5h,j572qtbd19x89mraje17mmj30x6sen0s,k176kkt806khrpxnctb9hczm316rtsrq,quizSubmission,js78b7hq6r6pk2c0mm9d2he91s6mjb8v,reading,"{'answers': [2.0, 3.0, 0.0], 'correctness': 1....",NaN,NaN,Dijkstra,A,A,Quiz #10,12.279168
3,2024-04-18 08:10:36.998193848,k57000m99awdk4jw41cjfe4jkn6qf1jj,j572w733907q5k2w0kkk4a19z96qfaec,k171kqa60g6a9qpm0f0jv07wn16ptnr1,quizSubmission,js7e56b2v2t5nacr7e4n8dnn3d6mb606,explain,"{'answers': [0.0, 1.0, 1.0], 'correctness': 0....",NaN,NaN,Breadth-First Search,B,A,Quiz #6,712.725918
4,2024-03-13 16:35:30.072217773,k57000mdpmrfm67q4eymvsbqns6n6ezf,j571e3e401tqj2mag6mjgrcynx6n7t96,k1715d99fyvzt35jfm2638g3nd6n35z3,exerciseCompleted,js7534h7gnm8pmyem0n6ttdb816mb94w,explain,NaN,NaN,NaN,Heap Sort,A,B,Quiz #3,9.432901


In [9]:
logs_grouped_messages = logs_ts.copy()
logs_grouped_messages = logs_grouped_messages[logs_grouped_messages["type"].isin(["messageSent", "answerGenerated","exerciseCompleted"])]
logs_grouped_messages["type"].value_counts()

type
messageSent          42307
answerGenerated      36761
exerciseCompleted     5063
Name: count, dtype: int64

In [10]:
logs_grouped_messages["n_sent"] = logs_grouped_messages.groupby(["userId", "exerciseName"])["type"].transform(lambda x: (x == "messageSent").sum())
logs_grouped_messages["n_sent"]= logs_grouped_messages["n_sent"].fillna(0).astype(int)

logs_grouped_messages["n_received"] = logs_grouped_messages.groupby(["userId", "exerciseName"])["type"].transform(lambda x: (x == "answerGenerated").sum())
logs_grouped_messages["n_received"]= logs_grouped_messages["n_received"].fillna(0).astype(int)

logs_grouped_messages["n_mess"] = logs_grouped_messages["n_sent"] + logs_grouped_messages["n_received"]
logs_grouped_messages


,_creationTime,_id,attemptId,exerciseId,type,userId,variant,details,systemMessageId,userMessageId,exerciseName,controlGroup,userGroup,week,timeSpent_quiz,n_sent,n_received,n_mess
4,2024-03-13 16:35:30.072217773,k57000mdpmrfm67q4eymvsbqns6n6ezf,j571e3e401tqj2mag6mjgrcynx6n7t96,k1715d99fyvzt35jfm2638g3nd6n35z3,exerciseCompleted,js7534h7gnm8pmyem0n6ttdb816mb94w,explain,NaN,NaN,NaN,Heap Sort,A,B,Quiz #3,9.432901,5,4,9
5,2024-03-28 16:24:01.600864014,k57000ny7w4g28vjeeyb9x8se16p55f9,j578ec52hh90fvkmk132x6ege96p45ba,k17asvnfeyp3q949hdkwjq2and6p2x6e,answerGenerated,js7984yjpe01cj1761gygtt3dx6mafdh,explain,NaN,jd7b55hm5qjm0196n9rt6jyh7n6p581t,jd70y58gcaafd916x0rfp0ygsh6p4b7a,Matrix-Chain Multiplication,A,B,Quiz #4,1132.250950,8,7,15
9,2024-04-04 13:43:46.323182861,k57000x8jdvr4gjsm2p0ey1bms6pj2vt,j57en6wxxqtd4x169vt90y9zdh6pkyh4,k1779x2fet2p0zkazg66nvfcjh6p2vbm,messageSent,js73xpgsqcv0gmv0g1w7as97rd6me4c0,explain,NaN,jd747sa89e37hf86z054b67wjx6pk19t,jd7b6sqaqbmgvxc1b14sr4rewn6pj0sj,Rod Cutting,B,A,Quiz #4,5.145569,9,8,17
10,2024-04-07 10:08:21.862825195,k5700154f6etndhepqx0rstrkn6pr0xk,j573anryyjzgxm9rcpdm6t9bys6psp8s,k17asvnfeyp3q949hdkwjq2and6p2x6e,messageSent,js74xtqm1gax5zqzszz0yrnfss6mekyy,explain,NaN,jd75ge56tp37t39fp4pw04mmcx6ps3s9,jd71yw479qb2cp2xv2k5b7z1n16psack,Matrix-Chain Multiplication,A,B,Quiz #4,1.805898,9,8,17
12,2024-04-22 09:28:34.208699707,k570015f8vcye8mbv9tb6w8hv96qpwq6,j57e15s47pzdb00vn5057zr6md6qq4aq,k17aatn2xs5xgqg6cgcjm1tqcn6q0cw7,messageSent,js71v2af0tfkjx57xy21wqhyyx6md2et,explain,NaN,jd71n1g8c1pd1vb0vgq841h0356qp18w,jd7dvv02ctfyw1p9ns9bd635j16qpydg,Depth-First Search,A,B,Quiz #6,9.959555,6,5,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150889,2024-04-17 20:09:57.378344482,k57fzz34c1ydkwt8w5648c7vxd6qd046,j57edg5teg7haqdw3y6jgtenkh6qcrfh,k17aatn2xs5xgqg6cgcjm1tqcn6q0cw7,answerGenerated,js72e99j78d0thce52z0w0594n6mbthj,explain,NaN,jd78jkknjzt33tmdx099zvtq1s6qc40m,jd76n4y5xnmpd32k0cctyb2mxx6qcesb,Depth-First Search,A,B,Quiz #6,9.773036,10,9,19
150890,2024-04-28 15:58:08.244800537,k57fzz4xh4a5j78rbrhpctvyxs6r2dh9,j574ym30w3hmatzk704wf785fx6r2ett,k172p4g9eanfgsgtkjg0sn49916qgwh4,answerGenerated,js7f8p4456h8p3f9am33n8p3f56mc1k2,explain,NaN,jd7btft8ad0g2v5j93jkz1qyw16r2xgb,jd709bqe9abqhft932cd7rymyn6r3qwj,Topological Sorting,B,A,Quiz #7,3.718620,9,8,17
150892,2024-04-04 09:59:25.539958252,k57fzzawztspx74s56dd8v9g7h6pkea6,j57cq176xgq49ebe4hq4ec4bb96pjfc1,k17asvnfeyp3q949hdkwjq2and6p2x6e,messageSent,js7ejegwwy13npt4t5xmxfaze16mf7wj,explain,NaN,jd765cr60qe2ygzfqsgq9950996pj5fm,jd77rttn3djr1548bzs6jp24hs6pkmyv,Matrix-Chain Multiplication,A,B,Quiz #4,6.870883,7,6,13
150894,2024-05-06 22:05:16.395833740,k57fzzdgj1jvxwbbb443hfkej96rkktd,j57ck5p09yecbjpse269svw8eh6rkapf,k1768n9arfqk6vpqtsaxgs3m156qx8mp,messageSent,js77jjffzjf9pskspa6hh9sknh6ma65a,explain,NaN,jd764p5pff8zfxjf67f3rb2rgn6rj0xv,jd7920e1qrchh4fj403hjw9xj16rkyqt,Edge disjoint paths via max flow,A,B,Quiz #8,24.978101,13,12,25


In [11]:
logs_grouped_messages["finish_time"] = logs_grouped_messages[logs_grouped_messages["type"]=="exerciseCompleted"]["_creationTime"]
logs_grouped_messages["start_time"] = logs_grouped_messages.sort_values("_creationTime").groupby(["userId", "exerciseName"])["_creationTime"].transform("min")
logs_grouped_messages["timeSpent_message"] = (logs_grouped_messages["finish_time"] - logs_grouped_messages["start_time"]).dt.total_seconds()/60

In [12]:
logs_grouped_messages = logs_grouped_messages.query("type=='exerciseCompleted'").drop_duplicates(subset=["userId","exerciseId"])[["userId","exerciseId", "n_mess", "timeSpent_message"]]
logs_grouped_messages

,userId,exerciseId,n_mess,timeSpent_message
4,js7534h7gnm8pmyem0n6ttdb816mb94w,k1715d99fyvzt35jfm2638g3nd6n35z3,9,3.208412
14,js76m5g9wh37e6px2tyb5psbvx6mg0tp,k177dxhnpwwtrq9dy5770jmrwx6rvzrr,11,2.332707
24,js788ecjy0bjd2azdgf9x43yk96mdat0,k171kqa60g6a9qpm0f0jv07wn16ptnr1,19,148.639481
60,js73t5rvpr3a90mdxdd8yvrzkn6mmp42,k17aatn2xs5xgqg6cgcjm1tqcn6q0cw7,27,731.854775
61,js75xfp2h55x14p9g4zhf2azw56mfxby,k173an7ns7g76j493paexdh88d6n3bsh,19,7.407522
...,...,...,...,...
150742,js7fevm1btqznxbqs17c8hmy0s6md575,k177dxhnpwwtrq9dy5770jmrwx6rvzrr,5,1.422896
150753,js74jr9c4cnqhjds24d44j4kmh6mb07r,k176kkt806khrpxnctb9hczm316rtsrq,11,4.005574
150767,js72s3x6qh9rd266x1pfg4baen6mhyd9,k17c95d62th7c5e5dq5b05s1ex6rjp81,15,8.160749
150862,js7636qxyx41pg59t5vt1se6j56mfnys,k17416m11n2qpqy5sr2zz6782s6s2y2n,13,22.265921


In [13]:
logs_ts2 = logs_ts.merge(logs_grouped_messages, how='left',on=['userId','exerciseId'])
logs_ts2

,_creationTime,_id,attemptId,exerciseId,type,userId,variant,details,systemMessageId,userMessageId,exerciseName,controlGroup,userGroup,week,timeSpent_quiz,n_mess,timeSpent_message
0,2024-05-11 15:56:52.688503174,k570005nhp144860betwtwy7v56rx64m,j571kc5p8ah7yvj1xhng7nzpgs6rxy2m,k179wpzex0za69rx2wyv02e9hd6rj4fb,attemptStarted,js71tk2nax2wvs97m3vgc5erp16mds09,reading,NaN,NaN,NaN,Kruskal's algorithm,A,A,Quiz #9,1.769433,NaN,NaN
1,2024-04-16 08:00:01.116921631,k57000a4rv5wgy13d63pqyt51s6qb2vv,j57adjsev6qjazakv9p4ad7qwh6qany3,k1710bmgh2mwgaqv7sjkmmfh1x6pveqf,quizSubmission,js7ese808vctnr15sy1te2pvx96mndmx,explain,"{'answers': [2.0, 0.0, 2.0], 'correctness': 1....",NaN,NaN,Optimal Binary Search Trees,A,B,Quiz #5,18.036219,11.0,16.125910
2,2024-05-20 16:28:14.092592285,k57000d308fpr45yqrprf6wrs56see5h,j572qtbd19x89mraje17mmj30x6sen0s,k176kkt806khrpxnctb9hczm316rtsrq,quizSubmission,js78b7hq6r6pk2c0mm9d2he91s6mjb8v,reading,"{'answers': [2.0, 3.0, 0.0], 'correctness': 1....",NaN,NaN,Dijkstra,A,A,Quiz #10,12.279168,NaN,NaN
3,2024-04-18 08:10:36.998193848,k57000m99awdk4jw41cjfe4jkn6qf1jj,j572w733907q5k2w0kkk4a19z96qfaec,k171kqa60g6a9qpm0f0jv07wn16ptnr1,quizSubmission,js7e56b2v2t5nacr7e4n8dnn3d6mb606,explain,"{'answers': [0.0, 1.0, 1.0], 'correctness': 0....",NaN,NaN,Breadth-First Search,B,A,Quiz #6,712.725918,13.0,13.314908
4,2024-03-13 16:35:30.072217773,k57000mdpmrfm67q4eymvsbqns6n6ezf,j571e3e401tqj2mag6mjgrcynx6n7t96,k1715d99fyvzt35jfm2638g3nd6n35z3,exerciseCompleted,js7534h7gnm8pmyem0n6ttdb816mb94w,explain,NaN,NaN,NaN,Heap Sort,A,B,Quiz #3,9.432901,9.0,3.208412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150892,2024-04-04 09:59:25.539958252,k57fzzawztspx74s56dd8v9g7h6pkea6,j57cq176xgq49ebe4hq4ec4bb96pjfc1,k17asvnfeyp3q949hdkwjq2and6p2x6e,messageSent,js7ejegwwy13npt4t5xmxfaze16mf7wj,explain,NaN,jd765cr60qe2ygzfqsgq9950996pj5fm,jd77rttn3djr1548bzs6jp24hs6pkmyv,Matrix-Chain Multiplication,A,B,Quiz #4,6.870883,13.0,10.088254
150893,2024-04-23 10:58:30.496422607,k57fzzbpy6sndtetjcsrfm8j7n6qrk4f,j573jw31r3p36xs9prfbskv8sn6qshfr,k17aatn2xs5xgqg6cgcjm1tqcn6q0cw7,feedbackGiven,js75phnd9pfy1mp9b3s4w94f0d6mjzaa,explain,NaN,jd70ajdywrabes973rbs30j3ax6qsnfk,NaN,Depth-First Search,A,B,Quiz #6,156.682048,35.0,891.539077
150894,2024-05-06 22:05:16.395833740,k57fzzdgj1jvxwbbb443hfkej96rkktd,j57ck5p09yecbjpse269svw8eh6rkapf,k1768n9arfqk6vpqtsaxgs3m156qx8mp,messageSent,js77jjffzjf9pskspa6hh9sknh6ma65a,explain,NaN,jd764p5pff8zfxjf67f3rb2rgn6rj0xv,jd7920e1qrchh4fj403hjw9xj16rkyqt,Edge disjoint paths via max flow,A,B,Quiz #8,24.978101,25.0,10.592223
150895,2024-04-18 12:31:21.107867188,k57fzzgqsqtqrqm43bfrgpazth6qfv3c,j573hc3tac12sym478a3f9634d6qe4yq,k17aatn2xs5xgqg6cgcjm1tqcn6q0cw7,attemptStarted,js77gaavvhwwnth677vgafz3sd6mbgt9,explain,NaN,NaN,NaN,Depth-First Search,A,B,Quiz #6,7.121689,13.0,4.801819


In [14]:
user_data = pd.read_json("DB_CS250_Snapshot/anonymized_student_data.jsonl",lines=True)
user_data

,identifier,gender,section,semester
0,409af2188426d31768163f9db90f57ee738aeaf0c761d4...,Male,IN,Bachelor semester 4
1,fd72fd0e1055b5f8ab7eabd631a84896575291b5592e66...,Male,SC,Bachelor semester 4
2,ba65e347859e087307971a7348a3b552fb655f9bf82abc...,Female,IN,Bachelor semester 4
3,43510d84df3d9289af7eb33da46ab0ebf212ab78107f69...,Female,IN,Bachelor semester 4
4,6538b7a72fae4173dbe8ba580e01145afaeb0f31b3f2f8...,Female,IN,Bachelor semester 4
...,...,...,...,...
459,d03cb94868d7249d51f13c0589bec349d36844fc006dcc...,Male,IF,Master semester 2
460,67d3e01046e32192d65b7665aefc5af529734c3e14cf83...,Male,SC_ECH,Spring semester
461,169293dc5d04fa93ed8074d949d7b13346ecf3a9888cc8...,Male,IN,Bachelor semester 4
462,384d32131aea1bcadaf651ea56feebe501b1f3c4bc29d7...,Male,IN,Bachelor semester 4


In [15]:
user_data["userData"] = user_data.apply(lambda x : {col: x[col] for col in user_data.columns if col != "identifier"}, axis=1)
user_data

,identifier,gender,section,semester,userData
0,409af2188426d31768163f9db90f57ee738aeaf0c761d4...,Male,IN,Bachelor semester 4,"{'gender': 'Male', 'section': 'IN', 'semester'..."
1,fd72fd0e1055b5f8ab7eabd631a84896575291b5592e66...,Male,SC,Bachelor semester 4,"{'gender': 'Male', 'section': 'SC', 'semester'..."
2,ba65e347859e087307971a7348a3b552fb655f9bf82abc...,Female,IN,Bachelor semester 4,"{'gender': 'Female', 'section': 'IN', 'semeste..."
3,43510d84df3d9289af7eb33da46ab0ebf212ab78107f69...,Female,IN,Bachelor semester 4,"{'gender': 'Female', 'section': 'IN', 'semeste..."
4,6538b7a72fae4173dbe8ba580e01145afaeb0f31b3f2f8...,Female,IN,Bachelor semester 4,"{'gender': 'Female', 'section': 'IN', 'semeste..."
...,...,...,...,...,...
459,d03cb94868d7249d51f13c0589bec349d36844fc006dcc...,Male,IF,Master semester 2,"{'gender': 'Male', 'section': 'IF', 'semester'..."
460,67d3e01046e32192d65b7665aefc5af529734c3e14cf83...,Male,SC_ECH,Spring semester,"{'gender': 'Male', 'section': 'SC_ECH', 'semes..."
461,169293dc5d04fa93ed8074d949d7b13346ecf3a9888cc8...,Male,IN,Bachelor semester 4,"{'gender': 'Male', 'section': 'IN', 'semester'..."
462,384d32131aea1bcadaf651ea56feebe501b1f3c4bc29d7...,Male,IN,Bachelor semester 4,"{'gender': 'Male', 'section': 'IN', 'semester'..."


In [16]:
userGrades = pd.read_json("DB_CS250_Snapshot/anonymized_grades.jsonl",lines=True)
userGrades["userGrades"] = userGrades.apply(lambda x : {col: x[col] for col in userGrades.columns if col != "identifier"}, axis=1)
userGrades

,identifier,coding,midterm,final,grade_out_of_100,grade_out_of_6,userGrades
0,409af2188426d31768163f9db90f57ee738aeaf0c761d4...,100.0,58,72,72.5,5.00,"{'coding': 100.0, 'midterm': 58, 'final': 72, ..."
1,fd72fd0e1055b5f8ab7eabd631a84896575291b5592e66...,100.0,47,46,53.5,4.00,"{'coding': 100.0, 'midterm': 47, 'final': 46, ..."
2,ba65e347859e087307971a7348a3b552fb655f9bf82abc...,100.0,84,82,86.2,5.50,"{'coding': 100.0, 'midterm': 84, 'final': 82, ..."
3,43510d84df3d9289af7eb33da46ab0ebf212ab78107f69...,100.0,50,49,59.5,4.25,"{'coding': 100.0, 'midterm': 50, 'final': 49, ..."
4,6538b7a72fae4173dbe8ba580e01145afaeb0f31b3f2f8...,100.0,90,70,82.0,5.25,"{'coding': 100.0, 'midterm': 90, 'final': 70, ..."
...,...,...,...,...,...,...,...
459,d03cb94868d7249d51f13c0589bec349d36844fc006dcc...,100.0,65,67,73.0,5.00,"{'coding': 100.0, 'midterm': 65, 'final': 67, ..."
460,67d3e01046e32192d65b7665aefc5af529734c3e14cf83...,100.0,87,86,89.1,5.75,"{'coding': 100.0, 'midterm': 87, 'final': 86, ..."
461,169293dc5d04fa93ed8074d949d7b13346ecf3a9888cc8...,0.0,40,46,46.0,3.50,"{'coding': 0.0, 'midterm': 40, 'final': 46, 'g..."
462,384d32131aea1bcadaf651ea56feebe501b1f3c4bc29d7...,85.0,84,79,83.2,5.50,"{'coding': 85.0, 'midterm': 84, 'final': 79, '..."


In [17]:
users2 = pd.read_json("DB_CS250_Snapshot/users/documents.jsonl",lines=True)
user_data_merged = user_data.merge(users2[["identifier","_id"]], on="identifier")
user_data_merged.rename(columns={"_id":"userId"}, inplace=True)
user_data_merged = user_data_merged.merge(userGrades[["identifier","userGrades"]], on="identifier")
user_data_merged


,identifier,gender,section,semester,userData,userId,userGrades
0,409af2188426d31768163f9db90f57ee738aeaf0c761d4...,Male,IN,Bachelor semester 4,"{'gender': 'Male', 'section': 'IN', 'semester'...",js7dgr0d8dd0wkwxpe4vta5s956mmymm,"{'coding': 100.0, 'midterm': 58, 'final': 72, ..."
1,fd72fd0e1055b5f8ab7eabd631a84896575291b5592e66...,Male,SC,Bachelor semester 4,"{'gender': 'Male', 'section': 'SC', 'semester'...",js7dvsm1vdajp3vq0mbpc7zkds6ms94f,"{'coding': 100.0, 'midterm': 47, 'final': 46, ..."
2,ba65e347859e087307971a7348a3b552fb655f9bf82abc...,Female,IN,Bachelor semester 4,"{'gender': 'Female', 'section': 'IN', 'semeste...",js78dkftemm1n5ms3xfybktrr16mbwve,"{'coding': 100.0, 'midterm': 84, 'final': 82, ..."
3,43510d84df3d9289af7eb33da46ab0ebf212ab78107f69...,Female,IN,Bachelor semester 4,"{'gender': 'Female', 'section': 'IN', 'semeste...",js7ezpapkygej698pz9yettmpn6mhsaw,"{'coding': 100.0, 'midterm': 50, 'final': 49, ..."
4,6538b7a72fae4173dbe8ba580e01145afaeb0f31b3f2f8...,Female,IN,Bachelor semester 4,"{'gender': 'Female', 'section': 'IN', 'semeste...",js7454x87sqbg8wfqgzet7dd8n6mg0jr,"{'coding': 100.0, 'midterm': 90, 'final': 70, ..."
...,...,...,...,...,...,...,...
458,d03cb94868d7249d51f13c0589bec349d36844fc006dcc...,Male,IF,Master semester 2,"{'gender': 'Male', 'section': 'IF', 'semester'...",js7cbyke1gannssqcev8cghe9n6mb60x,"{'coding': 100.0, 'midterm': 65, 'final': 67, ..."
459,67d3e01046e32192d65b7665aefc5af529734c3e14cf83...,Male,SC_ECH,Spring semester,"{'gender': 'Male', 'section': 'SC_ECH', 'semes...",js757rhzzwv21y6hqd6kafvtwx6mbsb6,"{'coding': 100.0, 'midterm': 87, 'final': 86, ..."
460,169293dc5d04fa93ed8074d949d7b13346ecf3a9888cc8...,Male,IN,Bachelor semester 4,"{'gender': 'Male', 'section': 'IN', 'semester'...",js793s2ssf0xctfnx7hn49jwbh6me0pm,"{'coding': 0.0, 'midterm': 40, 'final': 46, 'g..."
461,384d32131aea1bcadaf651ea56feebe501b1f3c4bc29d7...,Male,IN,Bachelor semester 4,"{'gender': 'Male', 'section': 'IN', 'semester'...",js73qg870qxv6dmtdvbkbx1k916mkt9h,"{'coding': 85.0, 'midterm': 84, 'final': 79, '..."


In [18]:
logs3 = logs_ts2.merge(user_data_merged[["userId","userData","userGrades"]],how="left", on="userId")
logs3

,_creationTime,_id,attemptId,exerciseId,type,userId,variant,details,systemMessageId,userMessageId,exerciseName,controlGroup,userGroup,week,timeSpent_quiz,n_mess,timeSpent_message,userData,userGrades
0,2024-05-11 15:56:52.688503174,k570005nhp144860betwtwy7v56rx64m,j571kc5p8ah7yvj1xhng7nzpgs6rxy2m,k179wpzex0za69rx2wyv02e9hd6rj4fb,attemptStarted,js71tk2nax2wvs97m3vgc5erp16mds09,reading,NaN,NaN,NaN,Kruskal's algorithm,A,A,Quiz #9,1.769433,NaN,NaN,"{'gender': 'Female', 'section': 'IN', 'semeste...","{'coding': 65.0, 'midterm': 98, 'final': 86, '..."
1,2024-04-16 08:00:01.116921631,k57000a4rv5wgy13d63pqyt51s6qb2vv,j57adjsev6qjazakv9p4ad7qwh6qany3,k1710bmgh2mwgaqv7sjkmmfh1x6pveqf,quizSubmission,js7ese808vctnr15sy1te2pvx96mndmx,explain,"{'answers': [2.0, 0.0, 2.0], 'correctness': 1....",NaN,NaN,Optimal Binary Search Trees,A,B,Quiz #5,18.036219,11.0,16.125910,"{'gender': 'Male', 'section': 'SC', 'semester'...","{'coding': 100.0, 'midterm': 63, 'final': 72, ..."
2,2024-05-20 16:28:14.092592285,k57000d308fpr45yqrprf6wrs56see5h,j572qtbd19x89mraje17mmj30x6sen0s,k176kkt806khrpxnctb9hczm316rtsrq,quizSubmission,js78b7hq6r6pk2c0mm9d2he91s6mjb8v,reading,"{'answers': [2.0, 3.0, 0.0], 'correctness': 1....",NaN,NaN,Dijkstra,A,A,Quiz #10,12.279168,NaN,NaN,"{'gender': 'Male', 'section': 'IN', 'semester'...","{'coding': 100.0, 'midterm': 55, 'final': 35, ..."
3,2024-04-18 08:10:36.998193848,k57000m99awdk4jw41cjfe4jkn6qf1jj,j572w733907q5k2w0kkk4a19z96qfaec,k171kqa60g6a9qpm0f0jv07wn16ptnr1,quizSubmission,js7e56b2v2t5nacr7e4n8dnn3d6mb606,explain,"{'answers': [0.0, 1.0, 1.0], 'correctness': 0....",NaN,NaN,Breadth-First Search,B,A,Quiz #6,712.725918,13.0,13.314908,"{'gender': 'Female', 'section': 'ING_MATH', 's...","{'coding': 100.0, 'midterm': 82, 'final': 81, ..."
4,2024-03-13 16:35:30.072217773,k57000mdpmrfm67q4eymvsbqns6n6ezf,j571e3e401tqj2mag6mjgrcynx6n7t96,k1715d99fyvzt35jfm2638g3nd6n35z3,exerciseCompleted,js7534h7gnm8pmyem0n6ttdb816mb94w,explain,NaN,NaN,NaN,Heap Sort,A,B,Quiz #3,9.432901,9.0,3.208412,"{'gender': 'Male', 'section': 'SC', 'semester'...","{'coding': 100.0, 'midterm': 87, 'final': 68, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150892,2024-04-04 09:59:25.539958252,k57fzzawztspx74s56dd8v9g7h6pkea6,j57cq176xgq49ebe4hq4ec4bb96pjfc1,k17asvnfeyp3q949hdkwjq2and6p2x6e,messageSent,js7ejegwwy13npt4t5xmxfaze16mf7wj,explain,NaN,jd765cr60qe2ygzfqsgq9950996pj5fm,jd77rttn3djr1548bzs6jp24hs6pkmyv,Matrix-Chain Multiplication,A,B,Quiz #4,6.870883,13.0,10.088254,"{'gender': 'Male', 'section': 'SC', 'semester'...","{'coding': 100.0, 'midterm': 60, 'final': 40, ..."
150893,2024-04-23 10:58:30.496422607,k57fzzbpy6sndtetjcsrfm8j7n6qrk4f,j573jw31r3p36xs9prfbskv8sn6qshfr,k17aatn2xs5xgqg6cgcjm1tqcn6q0cw7,feedbackGiven,js75phnd9pfy1mp9b3s4w94f0d6mjzaa,explain,NaN,jd70ajdywrabes973rbs30j3ax6qsnfk,NaN,Depth-First Search,A,B,Quiz #6,156.682048,35.0,891.539077,"{'gender': 'Female', 'section': 'SC', 'semeste...","{'coding': 0.0, 'midterm': 76, 'final': 79, 'g..."
150894,2024-05-06 22:05:16.395833740,k57fzzdgj1jvxwbbb443hfkej96rkktd,j57ck5p09yecbjpse269svw8eh6rkapf,k1768n9arfqk6vpqtsaxgs3m156qx8mp,messageSent,js77jjffzjf9pskspa6hh9sknh6ma65a,explain,NaN,jd764p5pff8zfxjf67f3rb2rgn6rj0xv,jd7920e1qrchh4fj403hjw9xj16rkyqt,Edge disjoint paths via max flow,A,B,Quiz #8,24.978101,25.0,10.592223,"{'gender': 'Male', 'section': 'IN', 'semester'...","{'coding': 50.0, 'midterm': 49, 'final': 58, '..."
150895,2024-04-18 12:31:21.107867188,k57fzzgqsqtqrqm43bfrgpazth6qfv3c,j573hc3tac12sym478a3f9634d6qe4yq,k17aatn2xs5xgqg6cgcjm1tqcn6q0cw7,attemptStarted,js77gaavvhwwnth677vgafz3sd6mbgt9,explain,NaN,NaN,NaN,Depth-First Search,A,B,Quiz #6,7.121689,13.0,4.801819,"{'gender': 'Male', 'section': 'IN', 'semester'...","{'coding': 65.0, 'midterm': 77, 'final': 87, '..."


In [19]:
users = pd.read_json(f"DB_CS250_Snapshot/users/documents.jsonl", lines=True)
with open(f"DB_CS250_Snapshot/valid_anonymized_identifiers.jsonl") as f:
    valid_users = eval(f.read())
valid_user_ids = []
for user in valid_users:
    try:
        valid_user_ids.append(users[users["identifier"] == user]["_id"].values[0])
    except (IndexError, KeyError):
        # Ignore the error and continue with the next iteration
        pass
print(valid_user_ids)
logs3["valid"]  = logs3["userId"].isin(valid_user_ids)
logs3

['js7b3v4zm7ck6gfrbszka2xrqd6mcafb', 'js79jxes9fvwncxyzgtb8twntx6mh60p', 'js7b1w7ynja0n6g5d1rr7c773s6ma3n7', 'js78kvh0vzc2c7eejs6cz61rdn6mhhgg', 'js74pf2m8e8jrz2gg9s1hep35s6mfngw', 'js74rtyskgxt8wmnz35fsjwt8h6mdw23', 'js725t4xf89kx3xkydecq6ahpn6mfse8', 'js76fbh084j5tdeaa55epwha356mfab2', 'js72fgshqwsrghh9nyhcz73dtx6mmzta', 'js72pp4cjyem9s6x0r4cm99hdx6mcq58', 'js7dvsm1vdajp3vq0mbpc7zkds6ms94f', 'js7edy4cz6zbqjg373e2rnm32d6mekbz', 'js7e3hs87vrmnnm9fzma135btn6mgd29', 'js76e9y9f04fgk56a4817d1qa56mbszv', 'js71yt7g8d5mvbykapvjn5x4a16mc32n', 'js71kg73z66z0d2zh94wraaeg96mqpbb', 'js71v2af0tfkjx57xy21wqhyyx6md2et', 'js7bewfegj93szsr9cqxe5hc6n6mkr96', 'js72sr0aj5f7bkpmysybmw48ph6mathd', 'js77jjffzjf9pskspa6hh9sknh6ma65a', 'js771zzv7xrf2tqrm119961kwd6mc1x2', 'js73z7rencz13vsq7a0g6ta8c96mcn7j', 'js76bctjnnch2rt1xwtx5dk5816mqwkr', 'js722d7691qcagwwxy6atxm5ps6mkt4c', 'js70c7dm9p3zn0bq2gkzmvdw6s6mqbfy', 'js73ezxcsgx591bne9tbzxqsm16mpsnf', 'js77c2682g7g4qg2d780garyss6mb6ee', 'js7fbyrrpw5pay6n0jhwn218d9

,_creationTime,_id,attemptId,exerciseId,type,userId,variant,details,systemMessageId,userMessageId,exerciseName,controlGroup,userGroup,week,timeSpent_quiz,n_mess,timeSpent_message,userData,userGrades,valid
0,2024-05-11 15:56:52.688503174,k570005nhp144860betwtwy7v56rx64m,j571kc5p8ah7yvj1xhng7nzpgs6rxy2m,k179wpzex0za69rx2wyv02e9hd6rj4fb,attemptStarted,js71tk2nax2wvs97m3vgc5erp16mds09,reading,NaN,NaN,NaN,Kruskal's algorithm,A,A,Quiz #9,1.769433,NaN,NaN,"{'gender': 'Female', 'section': 'IN', 'semeste...","{'coding': 65.0, 'midterm': 98, 'final': 86, '...",True
1,2024-04-16 08:00:01.116921631,k57000a4rv5wgy13d63pqyt51s6qb2vv,j57adjsev6qjazakv9p4ad7qwh6qany3,k1710bmgh2mwgaqv7sjkmmfh1x6pveqf,quizSubmission,js7ese808vctnr15sy1te2pvx96mndmx,explain,"{'answers': [2.0, 0.0, 2.0], 'correctness': 1....",NaN,NaN,Optimal Binary Search Trees,A,B,Quiz #5,18.036219,11.0,16.125910,"{'gender': 'Male', 'section': 'SC', 'semester'...","{'coding': 100.0, 'midterm': 63, 'final': 72, ...",True
2,2024-05-20 16:28:14.092592285,k57000d308fpr45yqrprf6wrs56see5h,j572qtbd19x89mraje17mmj30x6sen0s,k176kkt806khrpxnctb9hczm316rtsrq,quizSubmission,js78b7hq6r6pk2c0mm9d2he91s6mjb8v,reading,"{'answers': [2.0, 3.0, 0.0], 'correctness': 1....",NaN,NaN,Dijkstra,A,A,Quiz #10,12.279168,NaN,NaN,"{'gender': 'Male', 'section': 'IN', 'semester'...","{'coding': 100.0, 'midterm': 55, 'final': 35, ...",True
3,2024-04-18 08:10:36.998193848,k57000m99awdk4jw41cjfe4jkn6qf1jj,j572w733907q5k2w0kkk4a19z96qfaec,k171kqa60g6a9qpm0f0jv07wn16ptnr1,quizSubmission,js7e56b2v2t5nacr7e4n8dnn3d6mb606,explain,"{'answers': [0.0, 1.0, 1.0], 'correctness': 0....",NaN,NaN,Breadth-First Search,B,A,Quiz #6,712.725918,13.0,13.314908,"{'gender': 'Female', 'section': 'ING_MATH', 's...","{'coding': 100.0, 'midterm': 82, 'final': 81, ...",True
4,2024-03-13 16:35:30.072217773,k57000mdpmrfm67q4eymvsbqns6n6ezf,j571e3e401tqj2mag6mjgrcynx6n7t96,k1715d99fyvzt35jfm2638g3nd6n35z3,exerciseCompleted,js7534h7gnm8pmyem0n6ttdb816mb94w,explain,NaN,NaN,NaN,Heap Sort,A,B,Quiz #3,9.432901,9.0,3.208412,"{'gender': 'Male', 'section': 'SC', 'semester'...","{'coding': 100.0, 'midterm': 87, 'final': 68, ...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150892,2024-04-04 09:59:25.539958252,k57fzzawztspx74s56dd8v9g7h6pkea6,j57cq176xgq49ebe4hq4ec4bb96pjfc1,k17asvnfeyp3q949hdkwjq2and6p2x6e,messageSent,js7ejegwwy13npt4t5xmxfaze16mf7wj,explain,NaN,jd765cr60qe2ygzfqsgq9950996pj5fm,jd77rttn3djr1548bzs6jp24hs6pkmyv,Matrix-Chain Multiplication,A,B,Quiz #4,6.870883,13.0,10.088254,"{'gender': 'Male', 'section': 'SC', 'semester'...","{'coding': 100.0, 'midterm': 60, 'final': 40, ...",True
150893,2024-04-23 10:58:30.496422607,k57fzzbpy6sndtetjcsrfm8j7n6qrk4f,j573jw31r3p36xs9prfbskv8sn6qshfr,k17aatn2xs5xgqg6cgcjm1tqcn6q0cw7,feedbackGiven,js75phnd9pfy1mp9b3s4w94f0d6mjzaa,explain,NaN,jd70ajdywrabes973rbs30j3ax6qsnfk,NaN,Depth-First Search,A,B,Quiz #6,156.682048,35.0,891.539077,"{'gender': 'Female', 'section': 'SC', 'semeste...","{'coding': 0.0, 'midterm': 76, 'final': 79, 'g...",True
150894,2024-05-06 22:05:16.395833740,k57fzzdgj1jvxwbbb443hfkej96rkktd,j57ck5p09yecbjpse269svw8eh6rkapf,k1768n9arfqk6vpqtsaxgs3m156qx8mp,messageSent,js77jjffzjf9pskspa6hh9sknh6ma65a,explain,NaN,jd764p5pff8zfxjf67f3rb2rgn6rj0xv,jd7920e1qrchh4fj403hjw9xj16rkyqt,Edge disjoint paths via max flow,A,B,Quiz #8,24.978101,25.0,10.592223,"{'gender': 'Male', 'section': 'IN', 'semester'...","{'coding': 50.0, 'midterm': 49, 'final': 58, '...",True
150895,2024-04-18 12:31:21.107867188,k57fzzgqsqtqrqm43bfrgpazth6qfv3c,j573hc3tac12sym478a3f9634d6qe4yq,k17aatn2xs5xgqg6cgcjm1tqcn6q0cw7,attemptStarted,js77gaavvhwwnth677vgafz3sd6mbgt9,explain,NaN,NaN,NaN,Depth-First Search,A,B,Quiz #6,7.121689,13.0,4.801819,"{'gender': 'Male', 'section': 'IN', 'semester'...","{'coding': 65.0, 'midterm': 77, 'final': 87, '...",True


In [20]:
logs3.to_json("merged_logs.jsonl",orient="records",lines=True,date_format="iso")


In [21]:
#fill the nans with the existing values
logs3_filled = logs3.copy()
logs3_filled["timeSpent_message"] = logs3.groupby(["userId", "week"])["timeSpent_message"].transform(lambda x: x.fillna(x.mean()))
logs3_filled[logs3_filled["timeSpent_message"].isna()]

,_creationTime,_id,attemptId,exerciseId,type,userId,variant,details,systemMessageId,userMessageId,exerciseName,controlGroup,userGroup,week,timeSpent_quiz,n_mess,timeSpent_message,userData,userGrades,valid
82,2024-03-14 12:51:19.779082031,k57008e420zjrk4rqycsb3g8fh6n9jvw,j57apga7m4k16gfzx0kx7ea8ad6n80zf,k173an7ns7g76j493paexdh88d6n3bsh,quizSubmission,js726pqhe9kdaazj6n1jmvnmed6mntqk,reading,"{'answers': [2.0, 1.0, 2.0], 'correctness': 0....",NaN,NaN,Priority Queues,B,B,Quiz #3,NaN,NaN,NaN,"{'gender': 'Female', 'section': 'IN', 'semeste...","{'coding': 100.0, 'midterm': 61, 'final': 50, ...",True
241,2024-03-18 12:16:54.493211670,k5700rerp2m550eyr52fj1d21d6ngzgb,j57fxf2437d1x35491a2pasg2d6nhy2f,k173an7ns7g76j493paexdh88d6n3bsh,answerGenerated,js736926tyggf0htbbzhg1a6ad6mcqgz,explain,NaN,jd7cgzranz75krvy7h5h17bd596nhp9d,jd7erq8mmc7545z2wwew94x6wd6nhgy9,Priority Queues,B,A,Quiz #3,NaN,NaN,NaN,"{'gender': 'Male', 'section': 'SC', 'semester'...","{'coding': 0.0, 'midterm': 27, 'final': 63, 'g...",True
271,2024-04-11 11:15:33.415954590,k5700vkwga3cm4785ymg2w958n6q0321,j57egaecgtvrgbrk9x9jc7b02h6q1p92,k171kqa60g6a9qpm0f0jv07wn16ptnr1,attemptStarted,js741w5t33f39cpf8gpxsxv7gx6m6evh,explain,NaN,NaN,NaN,Breadth-First Search,B,A,Quiz #6,NaN,NaN,NaN,NaN,NaN,False
311,2024-04-22 23:49:50.132401367,k57010acdkh7jdmkcqdekd6b8n6qqy2x,j57e8pd4hfm1vrsyt3s5gqm3956qpz89,k171kqa60g6a9qpm0f0jv07wn16ptnr1,attemptStarted,js79d7mh1s0kng4774cf30rek56mcy2h,explain,NaN,NaN,NaN,Breadth-First Search,B,A,Quiz #6,NaN,NaN,NaN,"{'gender': 'Male', 'section': 'SC', 'semester'...","{'coding': 100.0, 'midterm': 74, 'final': 45, ...",True
383,2024-04-30 08:59:20.864013184,k57017xqr7qq3qgqk1xbg3s7vd6r7zmn,j57c00zhk8fntnpfancf1kn8nd6r6q0q,k172p4g9eanfgsgtkjg0sn49916qgwh4,attemptStarted,js7dg42jyqqd301gf9etf3sc156mann4,explain,NaN,NaN,NaN,Topological Sorting,B,A,Quiz #7,NaN,NaN,NaN,"{'gender': 'Male', 'section': 'MT_RO', 'semest...","{'coding': 0.0, 'midterm': 35, 'final': 47, 'g...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149403,2024-03-13 10:17:58.887569092,k57ftp2shqtx1y64fx9abbbcqs6n741v,j575m0db55tt3x4dbeq4j5c9a16n718w,k173an7ns7g76j493paexdh88d6n3bsh,attemptStarted,js77rjgaxfmzkb1q064wex2gk16mf99f,explain,NaN,NaN,NaN,Priority Queues,B,A,Quiz #3,NaN,NaN,NaN,NaN,NaN,False
149629,2024-03-01 09:18:58.341280029,k57fvd3e0k6krmc6h7x21srdwh6mfydp,j571pqx7q1sdg7banmch3y537d6mfjga,k17986at882ejjew66z1bgx5w96m67y2,messageSent,js75z2wr04v4kq0nhp9t6pexhd6mfvry,explain,NaN,jd71tjbfz1jw1cayv9zjadyg6s6mecbp,jd761swm3eg1628z7dxbvdf0v96me9tt,Insertion Sort,A,B,Quiz #1,NaN,NaN,NaN,"{'gender': 'Male', 'section': 'SC', 'semester'...","{'coding': 50.0, 'midterm': 43, 'final': 57, '...",True
149680,2024-03-14 10:58:45.134453125,k57fvjnzzbkx2a2cvd0x4x6hyx6n8qnb,j576bw2qw4kqypw42hbtrphg6n6n8t42,k1715d99fyvzt35jfm2638g3nd6n35z3,quizStarted,js78qw5bbj091w1xv4fwm3f6wx6ksj7p,reading,NaN,NaN,NaN,Heap Sort,A,A,Quiz #3,NaN,NaN,NaN,NaN,NaN,False
150145,2024-05-09 08:53:51.722703369,k57fx7fj7djgzd2t85zzd1ehgs6rs75b,j578zwrymwrmc4wbppdas820qs6rsb81,k179wpzex0za69rx2wyv02e9hd6rj4fb,answerGenerated,js7500r0mcptqpednexvk0y2696ms1sg,explain,NaN,jd74vatwagx4wanw92ga4gryq56rrx3v,jd71tkjravfbhj3a697dwvfgzn6rrmyj,Kruskal's algorithm,A,B,Quiz #9,NaN,NaN,NaN,"{'gender': 'Male', 'section': 'SC', 'semester'...","{'coding': 100.0, 'midterm': 74, 'final': 78, ...",True


In [22]:
logs_read = logs3_filled.copy()
start_time_read = logs_read[(logs_read["variant"]=='reading') &(logs_read["type"]=="attemptStarted")].groupby(["userId","week"])["_creationTime"].max().reset_index()
start_time_quiz = logs_read[(logs_read["variant"]=='reading') &(logs_read["type"]=="quizStarted")].groupby(["userId","week"])["_creationTime"].min().reset_index()
time_spent_read = start_time_read.merge(start_time_quiz, on=["userId","week"], how="left")
time_spent_read["timeSpent_read"] = (time_spent_read["_creationTime_y"] - time_spent_read["_creationTime_x"]).dt.total_seconds()/60
logs_read = logs_read.merge(time_spent_read[["userId","week","timeSpent_read"]], on=["userId","week"], how="left")
logs_read

,_creationTime,_id,attemptId,exerciseId,type,userId,variant,details,systemMessageId,userMessageId,...,controlGroup,userGroup,week,timeSpent_quiz,n_mess,timeSpent_message,userData,userGrades,valid,timeSpent_read
0,2024-05-11 15:56:52.688503174,k570005nhp144860betwtwy7v56rx64m,j571kc5p8ah7yvj1xhng7nzpgs6rxy2m,k179wpzex0za69rx2wyv02e9hd6rj4fb,attemptStarted,js71tk2nax2wvs97m3vgc5erp16mds09,reading,NaN,NaN,NaN,...,A,A,Quiz #9,1.769433,NaN,12.345785,"{'gender': 'Female', 'section': 'IN', 'semeste...","{'coding': 65.0, 'midterm': 98, 'final': 86, '...",True,7.015491
1,2024-04-16 08:00:01.116921631,k57000a4rv5wgy13d63pqyt51s6qb2vv,j57adjsev6qjazakv9p4ad7qwh6qany3,k1710bmgh2mwgaqv7sjkmmfh1x6pveqf,quizSubmission,js7ese808vctnr15sy1te2pvx96mndmx,explain,"{'answers': [2.0, 0.0, 2.0], 'correctness': 1....",NaN,NaN,...,A,B,Quiz #5,18.036219,11.0,16.125910,"{'gender': 'Male', 'section': 'SC', 'semester'...","{'coding': 100.0, 'midterm': 63, 'final': 72, ...",True,2.276635
2,2024-05-20 16:28:14.092592285,k57000d308fpr45yqrprf6wrs56see5h,j572qtbd19x89mraje17mmj30x6sen0s,k176kkt806khrpxnctb9hczm316rtsrq,quizSubmission,js78b7hq6r6pk2c0mm9d2he91s6mjb8v,reading,"{'answers': [2.0, 3.0, 0.0], 'correctness': 1....",NaN,NaN,...,A,A,Quiz #10,12.279168,NaN,14.979340,"{'gender': 'Male', 'section': 'IN', 'semester'...","{'coding': 100.0, 'midterm': 55, 'final': 35, ...",True,0.121073
3,2024-04-18 08:10:36.998193848,k57000m99awdk4jw41cjfe4jkn6qf1jj,j572w733907q5k2w0kkk4a19z96qfaec,k171kqa60g6a9qpm0f0jv07wn16ptnr1,quizSubmission,js7e56b2v2t5nacr7e4n8dnn3d6mb606,explain,"{'answers': [0.0, 1.0, 1.0], 'correctness': 0....",NaN,NaN,...,B,A,Quiz #6,712.725918,13.0,13.314908,"{'gender': 'Female', 'section': 'ING_MATH', 's...","{'coding': 100.0, 'midterm': 82, 'final': 81, ...",True,1.873318
4,2024-03-13 16:35:30.072217773,k57000mdpmrfm67q4eymvsbqns6n6ezf,j571e3e401tqj2mag6mjgrcynx6n7t96,k1715d99fyvzt35jfm2638g3nd6n35z3,exerciseCompleted,js7534h7gnm8pmyem0n6ttdb816mb94w,explain,NaN,NaN,NaN,...,A,B,Quiz #3,9.432901,9.0,3.208412,"{'gender': 'Male', 'section': 'SC', 'semester'...","{'coding': 100.0, 'midterm': 87, 'final': 68, ...",True,1.151624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150892,2024-04-04 09:59:25.539958252,k57fzzawztspx74s56dd8v9g7h6pkea6,j57cq176xgq49ebe4hq4ec4bb96pjfc1,k17asvnfeyp3q949hdkwjq2and6p2x6e,messageSent,js7ejegwwy13npt4t5xmxfaze16mf7wj,explain,NaN,jd765cr60qe2ygzfqsgq9950996pj5fm,jd77rttn3djr1548bzs6jp24hs6pkmyv,...,A,B,Quiz #4,6.870883,13.0,10.088254,"{'gender': 'Male', 'section': 'SC', 'semester'...","{'coding': 100.0, 'midterm': 60, 'final': 40, ...",True,0.116202
150893,2024-04-23 10:58:30.496422607,k57fzzbpy6sndtetjcsrfm8j7n6qrk4f,j573jw31r3p36xs9prfbskv8sn6qshfr,k17aatn2xs5xgqg6cgcjm1tqcn6q0cw7,feedbackGiven,js75phnd9pfy1mp9b3s4w94f0d6mjzaa,explain,NaN,jd70ajdywrabes973rbs30j3ax6qsnfk,NaN,...,A,B,Quiz #6,156.682048,35.0,891.539077,"{'gender': 'Female', 'section': 'SC', 'semeste...","{'coding': 0.0, 'midterm': 76, 'final': 79, 'g...",True,0.293077
150894,2024-05-06 22:05:16.395833740,k57fzzdgj1jvxwbbb443hfkej96rkktd,j57ck5p09yecbjpse269svw8eh6rkapf,k1768n9arfqk6vpqtsaxgs3m156qx8mp,messageSent,js77jjffzjf9pskspa6hh9sknh6ma65a,explain,NaN,jd764p5pff8zfxjf67f3rb2rgn6rj0xv,jd7920e1qrchh4fj403hjw9xj16rkyqt,...,A,B,Quiz #8,24.978101,25.0,10.592223,"{'gender': 'Male', 'section': 'IN', 'semester'...","{'coding': 50.0, 'midterm': 49, 'final': 58, '...",True,41.900725
150895,2024-04-18 12:31:21.107867188,k57fzzgqsqtqrqm43bfrgpazth6qfv3c,j573hc3tac12sym478a3f9634d6qe4yq,k17aatn2xs5xgqg6cgcjm1tqcn6q0cw7,attemptStarted,js77gaavvhwwnth677vgafz3sd6mbgt9,explain,NaN,NaN,NaN,...,A,B,Quiz #6,7.121689,13.0,4.801819,"{'gender': 'Male', 'section': 'IN', 'semester'...","{'coding': 65.0, 'midterm': 77, 'final': 87, '...",True,0.128499


In [23]:
messages = pd.read_json(f"{db}/messages/documents.jsonl", lines=True)
messages = messages[["content","_id","system"]].rename(columns={"_id":"systemMessageId"})
msg_sys = messages[messages["system"]]
msg_usr = messages[~messages["system"]]
logs_msg = logs_read.merge(msg_sys, on="systemMessageId", how="left")
logs_msg = logs_msg.merge(msg_usr, left_on="userMessageId",right_on="systemMessageId", how="left",suffixes=("","_usr"))
# regroup the columns
logs_msg.query("system == False")["content"] = np.nan
logs_msg = logs_msg.drop(columns=["systemMessageId_usr"])
logs_msg["system"] = logs_msg["system"].fillna(logs_msg["system_usr"])
logs_msg = logs_msg.drop(columns=["system_usr","system"])




In [24]:
logs_final = logs_msg.copy()
logs_final.to_json("merged_logs.jsonl",orient="records",lines=True,date_format="iso")

In [25]:
logs_final.columns

Index(['_creationTime', '_id', 'attemptId', 'exerciseId', 'type', 'userId',
       'variant', 'details', 'systemMessageId', 'userMessageId',
       'exerciseName', 'controlGroup', 'userGroup', 'week', 'timeSpent_quiz',
       'n_mess', 'timeSpent_message', 'userData', 'userGrades', 'valid',
       'timeSpent_read', 'content', 'content_usr'],
      dtype='object')

In [26]:
logs_csv = logs_final.copy()
logs_csv = logs_csv[logs_csv["valid"]]

In [27]:
logs_csv["userData"]

0         {'gender': 'Female', 'section': 'IN', 'semeste...
1         {'gender': 'Male', 'section': 'SC', 'semester'...
2         {'gender': 'Male', 'section': 'IN', 'semester'...
3         {'gender': 'Female', 'section': 'ING_MATH', 's...
4         {'gender': 'Male', 'section': 'SC', 'semester'...
                                ...                        
150892    {'gender': 'Male', 'section': 'SC', 'semester'...
150893    {'gender': 'Female', 'section': 'SC', 'semeste...
150894    {'gender': 'Male', 'section': 'IN', 'semester'...
150895    {'gender': 'Male', 'section': 'IN', 'semester'...
150896    {'gender': 'Female', 'section': 'IN', 'semeste...
Name: userData, Length: 128422, dtype: object

In [28]:
for key in logs_csv["userData"][0].keys():
    logs_csv[key] = logs_csv["userData"].apply(lambda x: x[key] if type(x)==dict else np.nan)
logs_csv.columns

Index(['_creationTime', '_id', 'attemptId', 'exerciseId', 'type', 'userId',
       'variant', 'details', 'systemMessageId', 'userMessageId',
       'exerciseName', 'controlGroup', 'userGroup', 'week', 'timeSpent_quiz',
       'n_mess', 'timeSpent_message', 'userData', 'userGrades', 'valid',
       'timeSpent_read', 'content', 'content_usr', 'gender', 'section',
       'semester'],
      dtype='object')

In [29]:
for key in logs_csv["userGrades"][0].keys():
    logs_csv[key] = logs_csv["userGrades"].apply(lambda x: x[key] if type(x)==dict else np.nan)
logs_csv.columns

Index(['_creationTime', '_id', 'attemptId', 'exerciseId', 'type', 'userId',
       'variant', 'details', 'systemMessageId', 'userMessageId',
       'exerciseName', 'controlGroup', 'userGroup', 'week', 'timeSpent_quiz',
       'n_mess', 'timeSpent_message', 'userData', 'userGrades', 'valid',
       'timeSpent_read', 'content', 'content_usr', 'gender', 'section',
       'semester', 'coding', 'midterm', 'final', 'grade_out_of_100',
       'grade_out_of_6'],
      dtype='object')

In [30]:

logs_csv["coding"].describe()

count    128422.000000
mean         83.391417
std          31.282660
min           0.000000
25%          85.000000
50%         100.000000
75%         100.000000
max         100.000000
Name: coding, dtype: float64

In [31]:
logs_csv["n_attempts"] = logs_csv.groupby(["userId","exerciseName"])["type"].transform(lambda x: (x=='quizSubmission').sum())


In [32]:
logs_csv = logs_csv.drop_duplicates(subset=["userId","exerciseId"])
logs_csv

,_creationTime,_id,attemptId,exerciseId,type,userId,variant,details,systemMessageId,userMessageId,...,content_usr,gender,section,semester,coding,midterm,final,grade_out_of_100,grade_out_of_6,n_attempts
0,2024-05-11 15:56:52.688503174,k570005nhp144860betwtwy7v56rx64m,j571kc5p8ah7yvj1xhng7nzpgs6rxy2m,k179wpzex0za69rx2wyv02e9hd6rj4fb,attemptStarted,js71tk2nax2wvs97m3vgc5erp16mds09,reading,NaN,NaN,NaN,...,NaN,Female,IN,Bachelor semester 4,65.0,98,86,88.9,5.75,1
1,2024-04-16 08:00:01.116921631,k57000a4rv5wgy13d63pqyt51s6qb2vv,j57adjsev6qjazakv9p4ad7qwh6qany3,k1710bmgh2mwgaqv7sjkmmfh1x6pveqf,quizSubmission,js7ese808vctnr15sy1te2pvx96mndmx,explain,"{'answers': [2.0, 0.0, 2.0], 'correctness': 1....",NaN,NaN,...,NaN,Male,SC,Bachelor semester 4,100.0,63,72,74.0,5.00,9
2,2024-05-20 16:28:14.092592285,k57000d308fpr45yqrprf6wrs56see5h,j572qtbd19x89mraje17mmj30x6sen0s,k176kkt806khrpxnctb9hczm316rtsrq,quizSubmission,js78b7hq6r6pk2c0mm9d2he91s6mjb8v,reading,"{'answers': [2.0, 3.0, 0.0], 'correctness': 1....",NaN,NaN,...,NaN,Male,IN,Bachelor semester 4,100.0,55,35,51.3,3.75,7
3,2024-04-18 08:10:36.998193848,k57000m99awdk4jw41cjfe4jkn6qf1jj,j572w733907q5k2w0kkk4a19z96qfaec,k171kqa60g6a9qpm0f0jv07wn16ptnr1,quizSubmission,js7e56b2v2t5nacr7e4n8dnn3d6mb606,explain,"{'answers': [0.0, 1.0, 1.0], 'correctness': 0....",NaN,NaN,...,NaN,Female,ING_MATH,Master semester 2,100.0,82,81,85.1,5.50,53
4,2024-03-13 16:35:30.072217773,k57000mdpmrfm67q4eymvsbqns6n6ezf,j571e3e401tqj2mag6mjgrcynx6n7t96,k1715d99fyvzt35jfm2638g3nd6n35z3,exerciseCompleted,js7534h7gnm8pmyem0n6ttdb816mb94w,explain,NaN,NaN,NaN,...,NaN,Male,SC,Bachelor semester 6,100.0,87,68,79.2,5.25,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132819,2024-04-26 21:43:45.448049316,k57e2tnp05btvz259wne7sdzv16qzmg5,j575dm96pgm5qpznym59h7wsgh6qympd,k172p4g9eanfgsgtkjg0sn49916qgwh4,attemptStarted,js7dfm6gbhfnw838g22wjynnr96mmmm5,reading,NaN,NaN,NaN,...,NaN,Male,SC,Bachelor semester 4,100.0,73,52,67.0,4.50,0
134554,2024-05-15 13:59:11.622167969,k57e8q1hhjynp478fnbx4sy7jd6s4eh4,j578y7jesez3j110r5epzmp9fn6s4h2p,k177dxhnpwwtrq9dy5770jmrwx6rvzrr,quizStarted,js72e99j78d0thce52z0w0594n6mbthj,reading,NaN,NaN,NaN,...,NaN,Male,MATH,Master semester 2,65.0,77,62,70.6,4.75,1
134867,2024-05-27 14:11:33.026981689,k57e9nzc3x0dhvhkqb5jv3xhb56sw770,j57a5g4ckqkmjnxfz33xngkxkh6sxzzw,k17416m11n2qpqy5sr2zz6782s6s2y2n,attemptStarted,js71v2af0tfkjx57xy21wqhyyx6md2et,reading,NaN,NaN,NaN,...,NaN,Male,IN,Bachelor semester 4,100.0,72,51,67.1,4.50,1
136952,2024-04-06 16:13:04.526881104,k57egqzbekxz2cx13fqarx8jfx6pqg7t,j57bb2w2jk67bjny2fpfrw349s6pqmt1,k1779x2fet2p0zkazg66nvfcjh6p2vbm,attemptStarted,js74bee168c3m9qtpwj7fjjt496mgx7x,reading,NaN,NaN,NaN,...,NaN,Male,IN,Bachelor semester 6,100.0,54,49,59.8,4.25,0


In [33]:
logs_csv.columns

Index(['_creationTime', '_id', 'attemptId', 'exerciseId', 'type', 'userId',
       'variant', 'details', 'systemMessageId', 'userMessageId',
       'exerciseName', 'controlGroup', 'userGroup', 'week', 'timeSpent_quiz',
       'n_mess', 'timeSpent_message', 'userData', 'userGrades', 'valid',
       'timeSpent_read', 'content', 'content_usr', 'gender', 'section',
       'semester', 'coding', 'midterm', 'final', 'grade_out_of_100',
       'grade_out_of_6', 'n_attempts'],
      dtype='object')

In [34]:
column_list = [
    "_creationTime",
    "_id",
    "variant",
    "userId",
    "exerciseId",
    "exerciseName",
    "week",
    "controlGroup",
    "userGroup",
    "n_attempts",
    "timeSpent_quiz",
    "n_mess",
    "timeSpent_message",
    "timeSpent_read",
    'gender',
    'section',
    'semester',
    'coding',
    'midterm',
    'final',
    'grade_out_of_100',
    'grade_out_of_6'
    ]

In [35]:
logs_csv = logs_csv[column_list].sort_values(["userId","_creationTime"])
logs_csv.to_csv("merged_logs.csv",index=False)